In [1]:
#!pip install tflearn
#!pip install tensorflow

In [2]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
from tensorflow.python.framework import ops
import numpy as np
import tflearn
import tensorflow as tf
import random

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [ ]:
nltk.download('punkt')
from nltk import sent_tokenize

In [4]:
# import our chat-bot intents file
import json
with open('data.json') as json_data:
    intents = json.load(json_data)

In [5]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

41 documents
12 classes ['Bye', 'Name', 'School subjects', 'Thanks', 'choose', 'division', 'final decision', 'greeting', 'help', 'orientation 1 year baccalaureate', 'prospects', 'understand']
115 unique stemmed words ['!', "'d", "'m", "'s", 'a', 'about', 'adv', 'al', 'am', 'and', 'any', 'ar', 'assist', 'b', 'bear', 'becaus', 'between', 'but', 'bye', 'ca', 'cal', 'can', 'cant', 'car', 'cho', 'choos', 'concern', 'confus', 'conveny', 'could', 'day', 'decid', 'did', 'dilemm', 'divid', 'dont', 'engin', 'fac', 'for', 'get', 'good', 'hav', 'hello', 'help', 'hey', 'hi', 'how', 'i', 'im', 'in', 'inform', 'interest', 'into', 'is', 'it', 'lat', 'leav', 'lik', 'lot', 'lov', 'main', 'mak', 'many', 'math', 'mathem', 'me', 'much', 'my', "n't", 'nam', 'nee', 'not', 'oh', 'okay', 'on', 'opt', 'or', 'ory', 'phys', 'pick', 'problem', 'prospect', 'provid', 'puzzl', 'regard', 'said', 'sci', 'see', 'simil', 'so', 'solv', 'som', 'soon', 'spec', 'stil', 'stud', 'suit', 'sur', 'tak', 'thank', 'that', 'the', 't

In [6]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

<ipython-input-6-65cc626bc989>:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [7]:
# reset underlying graph data
ops.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 5999  | total loss: 0.40595 | time: 0.021s
| Adam | epoch: 1000 | loss: 0.40595 - acc: 0.9765 -- iter: 40/41
Training Step: 6000  | total loss: 0.36554 | time: 0.026s
| Adam | epoch: 1000 | loss: 0.36554 - acc: 0.9788 -- iter: 41/41
--
INFO:tensorflow:C:\Users\saadm\Desktop\chatbot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:C:\Users\saadm\Desktop\chatbot\model.tflearn.data-00000-of-00001
INFO:tensorflow:0
INFO:tensorflow:C:\Users\saadm\Desktop\chatbot\model.tflearn.index
INFO:tensorflow:0
INFO:tensorflow:C:\Users\saadm\Desktop\chatbot\model.tflearn.meta
INFO:tensorflow:100


In [8]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [9]:
p = bow("Name", words)
print (p)
print (classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0]
['Bye', 'Name', 'School subjects', 'Thanks', 'choose', 'division', 'final decision', 'greeting', 'help', 'orientation 1 year baccalaureate', 'prospects', 'understand']


In [10]:
print(model.predict([p]))

[[2.0943597e-05 6.6024786e-01 1.2564571e-01 4.7005680e-03 1.1676166e-01
  5.3395644e-02 1.5208693e-02 5.0595059e-04 1.1426690e-02 1.4152908e-04
  1.1761426e-02 1.8338434e-04]]


In [11]:
# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

In [13]:
# restore all of our data structures
import pickle
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# import our chat-bot intents file
import json
with open('data.json') as json_data:
    intents = json.load(json_data)

In [14]:
# load our saved model
model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from C:\Users\saadm\Desktop\chatbot\model.tflearn


In [15]:
ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    return print(random.choice(i['responses']))

            results.pop(0)

In [ ]:
i=0
while i==0:
    inpuut=input("user: ")
    if inpuut!="":
        print('Agent: ',end="")
        response(inpuut)
    else:
        i=1

user: hello
Agent: Hello! sir
user: i'm saad
Agent: Hi there, How can i help you ?
